In [5]:
!pip install translate
!pip install langdetect

In [6]:
import pandas as pd
import json

with open('/content/transcripts.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data into a list of dictionaries
data_list = [{"ID": key, "Description": value} for key, value in data.items()]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)
df

,ID,Description
0,2055,"During the visit, I examined Mr. Don Hicks, wh..."
1,291,"During the visit, I examined Tina Will, a 69-y..."
2,102,"D: Good morning Tommie, how can I help you tod..."
3,2966,"D: Good morning, Chris. I understand you've be..."
4,2438,"D: Hi Ernest, I understand you're here for a c..."
...,...,...
1996,1732,"D: ¡Buenos días, Erwin Thompson! Entiendo que ..."
1997,2193,"During the visit, I asked Ms. Diana Griffith a..."
1998,2833,"During the visit, I, the doctor, explained to ..."
1999,1221,"Durin' the vissit, Mildred Favero, aged 93, pr..."


In [7]:
from langdetect import detect  # Import the language detection library
from translate import Translator

# Function to translate text with error handling
def translate_text(text):
    try:
        translator = Translator(to_lang="en", from_lang="auto")
        translation = translator.translate(text)
        return translation
    except Exception as e:
        print(f"Error during translation: {e}")
        return text  # Return the original text for other errors

# Function to check if text is already in English
def is_english(text):
    try:
        detected_lang = detect(text)
        return detected_lang == "en"
    except Exception as e:
        print(f"Error during language detection: {e}")
        return False  # Assume it's not in English if there's an error

# Translate each description in the DataFrame (only if not already in English)
df['Translated_Description'] = df['Description'].apply(lambda text: text if is_english(text) else translate_text(text))

# Print the DataFrame with translated descriptions
print(df)


        ID                                        Description  \
0     2055  During the visit, I examined Mr. Don Hicks, wh...   
1      291  During the visit, I examined Tina Will, a 69-y...   
2      102  D: Good morning Tommie, how can I help you tod...   
3     2966  D: Good morning, Chris. I understand you've be...   
4     2438  D: Hi Ernest, I understand you're here for a c...   
...    ...                                                ...   
1996  1732  D: ¡Buenos días, Erwin Thompson! Entiendo que ...   
1997  2193  During the visit, I asked Ms. Diana Griffith a...   
1998  2833  During the visit, I, the doctor, explained to ...   
1999  1221  Durin' the vissit, Mildred Favero, aged 93, pr...   
2000  1284  D: Good morning, Mr. Burns. How can I help you...   

                                 Translated_Description  
0     During the visit, I examined Mr. Don Hicks, wh...  
1     During the visit, I examined Tina Will, a 69-y...  
2     D: Good morning Tommie, how can I help 

In [8]:
df.to_csv('transcripts_translated.csv',index=False)

In [9]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spacy
from transformers import pipeline

# Load an NLP model (e.g., spaCy)
nlp = spacy.load("en_core_web_sm")

# Load a GPT-based summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Load a GPT-based text generation model (for rephrasing)
text_generator = pipeline("text-generation", model="gpt2")

# Function to process and validate medical information
def process_medical_info(transcript):
    # Extract relevant information using spaCy or custom methods
    doc = nlp(transcript)
    extracted_info = {}  # Store extracted information here

    # Rephrase extracted information using the text generation model
    rephrased_info = text_generator(transcript, max_length=50, num_return_sequences=1)
    rephrased_info = rephrased_info[0]['generated_text']

    # Summarize the transcript
    transcript_list = [transcript]  # Convert the transcript to a list
    summary = summarizer(transcript_list, max_length=100, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = summary[0]['summary_text']

    # Additional validation and processing steps
    # ...

    return {
        "extracted_info": extracted_info,
        "rephrased_info": rephrased_info,
        "summary": summary
    }

# Process each transcript and store the results
processed_data = []

for transcript in df['Description']:
    result = process_medical_info(transcript)
    processed_data.append(result)

# Create a new DataFrame to store the processed data
processed_df = pd.DataFrame(processed_data)

# Save the processed data to a new JSON file
processed_df.to_json('/content/processed_transcripts.json', orient='records')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 102, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 104, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 298, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
S

In [ ]:
processed_df